# Resources 
- [Data Source](https://8weeksqlchallenge.com/case-study-2/)

# Sub-querying 
- SELECT Clause 
- WHERE clause
- FROM clause
- Common Table Expressions

## SELECT Clause Sub-querying 
- Return each order run along with its ranking based on distance traveled. The further the distance, the higher the ranking
- Get the percentage of the total distance travelled that a specific order made up
- Get the registration date with the runner order without using a join

In [30]:
-- Get the ranking of each distance
SELECT order_id, runner_id, distance, (
    SELECT COUNT(*) + 1
    FROM runner_orders
    WHERE distance > r.distance
)
FROM runner_orders AS r

(10 row(s) affected)

Total execution time: 00:00:00.026

order_id,runner_id,distance,Column4
3,1,13.4,6
4,2,23.4,2
5,3,10,7
1,1,20,4
2,1,20,4
7,2,25,1
8,2,23.4,2
10,1,10,7
6,3,NULL,1
9,2,NULL,1


In [31]:
-- Get the percentage of the total distance travelled that a specific order made up
SELECT order_id, runner_id, distance, (
    SELECT (SUM(distance) * r.distance) / 100
    FROM runner_orders
) AS "Distance Percentage"
FROM runner_orders AS r

(10 row(s) affected)

Total execution time: 00:00:00.016

order_id,runner_id,distance,Distance Percentage
3,1,13.4,19.456799999999998
4,2,23.4,33.9768
5,3,10,14.52
1,1,20,29.04
2,1,20,29.04
7,2,25,36.3
8,2,23.4,33.9768
10,1,10,14.52
6,3,NULL,NULL
9,2,NULL,NULL


In [32]:
-- Get the registration date with the runner order without using a join
SELECT order_id, runner_id, distance, (
    SELECT registration_date 
    FROM runners
    WHERE r.runner_id = runner_id
) 
FROM runner_orders AS r

(10 row(s) affected)

Total execution time: 00:00:00.017

order_id,runner_id,distance,registration_date
3,1,13.4,2021-01-01
4,2,23.4,2021-01-03
5,3,10,2021-01-08
1,1,20,2021-01-01
2,1,20,2021-01-01
7,2,25,2021-01-03
8,2,23.4,2021-01-03
10,1,10,2021-01-01
6,3,NULL,2021-01-08
9,2,NULL,2021-01-03


## Where Clause Sub-querying 
- Get all of the orders that have been cancelled 
- Get all orders that wer further than the average distance 
- Return the order that had the furthest distance 

In [33]:
-- Get all of the orders that have been cancelled
SELECT * 
FROM customer_orders AS co
WHERE order_id IN (
    SELECT order_id 
    FROM runner_orders
    WHERE cancellation IS NOT NULL    
)

(2 row(s) affected)

Total execution time: 00:00:00.014

order_id,customer_id,pizza_id,exclusions,extras,order_time
9,103,1,4,"1, 5",2020-01-10 11:22:59
6,101,2,NULL,NULL,2020-01-08 21:03:13


In [40]:
-- Get furthest distance
SELECT * 
FROM customer_orders AS co
WHERE order_id = (
    SELECT order_id
    FROM runner_orders
    ORDER BY distance DESC
    LIMIT 1
)

(1 row(s) affected)

Total execution time: 00:00:00.021

order_id,customer_id,pizza_id,exclusions,extras,order_time
6,101,2,NULL,NULL,2020-01-08 21:03:13


## FROM Clause sub-querying
- Get the percentage of the total distance that an order made up, only show order that made up 10% or more of the total distance
- Get the names of the last 5 pizzas that got cancelled
- Get the name of the Pizza that gets cancelled the most

In [53]:
-- Get the difference between the pickup and order times for each order
SELECT order_id, distance, pickup_time - order_time
FROM (
    SELECT CO.order_id, customer_id, distance, pickup_time, order_time
    FROM customer_orders AS CO
    INNER JOIN runner_orders
    USING(order_id)
) AS order_runner_time

(14 row(s) affected)

Total execution time: 00:00:00.048

order_id,distance,Column3
9,NULL,NULL
10,10,00:15:31
4,23.4,00:29:17
4,23.4,00:29:17
4,23.4,00:29:17
5,10,00:10:28
7,25,00:10:16
6,NULL,NULL
8,23.4,00:20:29
10,10,00:15:31


## Common Table Expressions 
- Use CTE's for the statements from the `FROM` clause examples

In [47]:
WITH order_runner_time AS (
    SELECT CO.order_id, customer_id, distance, pickup_time, order_time
    FROM customer_orders AS CO
    INNER JOIN runner_orders
    USING(order_id)
)
SELECT order_id, distance
FROM order_runner_time


(14 row(s) affected)

Total execution time: 00:00:00.021

order_id,distance
9,NULL
10,10
4,23.4
4,23.4
4,23.4
5,10
7,25
6,NULL
8,23.4
10,10


# Aggregation
- GROUP BY clause 
- HAVING clause 

## GROUP BY clause 
- Get the average distance travelled 
- Get the greatest and shortest distances travelled 
- Get the number of orders done on each day

In [49]:
SELECT AVG(distance) AS average_distance
FROM runner_orders

(1 row(s) affected)

Total execution time: 00:00:00.026

average_distance
18.15


In [58]:
-- Get the average distance travelled per day 
SELECT EXTRACT(DAY FROM pickup_time) AS "the day", AVG(distance) AS average_distance
FROM runner_orders
GROUP BY EXTRACT(DAY FROM pickup_time)

(7 row(s) affected)

Total execution time: 00:00:00.028

the day,average_distance
NULL,NULL
3,13.4
1,20
4,23.4
10,23.4
8,17.5
11,10


## HAVING clause 
- Get the days that had more than 2 orders 
- Get the runners with the most orders 

In [61]:
-- Get the average distance travelled per day 
SELECT EXTRACT(DAY FROM pickup_time) AS "the day", AVG(distance) AS average_distance
FROM runner_orders
GROUP BY EXTRACT(DAY FROM pickup_time)
HAVING AVG(distance) < 15

(2 row(s) affected)

Total execution time: 00:00:00.007

the day,average_distance
3,13.4
11,10
